# Get Files

In [9]:
import sys
import glob
Filelist = glob.glob('<full path to OUTPUT files>\OUTPUT*.txt')

# ordering files
Filelist.insert(0, Filelist.pop())
print(Filelist)

['C:\\Users\\Milo Shan\\Desktop\\Sprinkler Stuff\\Sprinkler Data\\OUTPUT.txt', 'C:\\Users\\Milo Shan\\Desktop\\Sprinkler Stuff\\Sprinkler Data\\OUTPUT 2.txt', 'C:\\Users\\Milo Shan\\Desktop\\Sprinkler Stuff\\Sprinkler Data\\OUTPUT 3.TXT', 'C:\\Users\\Milo Shan\\Desktop\\Sprinkler Stuff\\Sprinkler Data\\OUTPUT 4.TXT', 'C:\\Users\\Milo Shan\\Desktop\\Sprinkler Stuff\\Sprinkler Data\\OUTPUT 5.TXT', 'C:\\Users\\Milo Shan\\Desktop\\Sprinkler Stuff\\Sprinkler Data\\OUTPUT 6.TXT', 'C:\\Users\\Milo Shan\\Desktop\\Sprinkler Stuff\\Sprinkler Data\\OUTPUT 7.TXT', 'C:\\Users\\Milo Shan\\Desktop\\Sprinkler Stuff\\Sprinkler Data\\OUTPUT 8.TXT']


In [10]:
import pandas as pd

dfList = [pd.read_csv(file) for file in Filelist]

for df in dfList[:3]:
    df['wind'] *= 4

In [11]:
df = pd.concat(dfList, ignore_index = True)

In [12]:
df.shape

(246232, 8)

# Smooth

In [13]:
%matplotlib widget
import numpy

def smooth(x,window_len=11,window='hanning'):
    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal
        
    example:

    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)
    
    see also: 
    
    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter
 
    TODO: the window parameter could be the window itself if an array instead of a string
    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
    """

    if x.ndim != 1:
        raise ValueError("smooth only accepts 1 dimension arrays.")

    if x.size < window_len:
        raise ValueError("Input vector needs to be bigger than window size.")


    if window_len<3:
        return x


    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError("Window is one of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")


    s=numpy.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    #print(len(s))
    if window == 'flat': #moving average
        w=numpy.ones(window_len,'d')
    else:
        w=eval('numpy.'+window+'(window_len)')

    y=numpy.convolve(w/w.sum(),s,mode='valid')
    return y[(window_len//2):-(window_len//2)]

df.airHum = smooth(df.airHum,71,'blackman')
df.airTemp = smooth(df.airTemp,71,'blackman')
df.soilTemp = smooth(df.soilTemp,51,'flat')
df.soilHum = smooth(df.soilHum,91,'blackman')
df.wind = smooth(df.wind,81,'blackman')

# Rainfall Data Setup & Merging

In [14]:
# Set up rainfall data
df2 = pd.read_csv('<full path to Rainfal data file>\Midstream Estate Rainfall Data.csv')
df2 = df2.dropna()
df2

,DATE,AA1
274,2021-08-31 12:00:00,"06,0010,3,1"
300,2021-09-04 00:00:00,"18,0006,3,1"
305,2021-09-04 12:00:00,"06,0420,3,1"
313,2021-09-05 12:00:00,"06,0360,3,1"
315,2021-09-05 18:00:00,"06,1450,3,1"
...,...,...
1482,2022-02-05 06:00:00,"06,0160,3,1"
1508,2022-02-09 06:00:00,"24,0030,3,1"
1514,2022-02-10 00:00:00,"18,0250,3,1"
1518,2022-02-10 12:00:00,"06,0002,3,1"


In [15]:
# convert to unixtime
import time
import datetime
for x in df2["DATE"]:
    df2 = df2.replace(x, time.mktime(datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").timetuple()))
for x in df2['AA1']:
    df2 = df2.replace(x, ','.join(x.split(',')[:2]))
df2

,DATE,AA1
274,1.630404e+09,"06,0010"
300,1.630706e+09,"18,0006"
305,1.630750e+09,"06,0420"
313,1.630836e+09,"06,0360"
315,1.630858e+09,"06,1450"
...,...,...
1482,1.644034e+09,"06,0160"
1508,1.644379e+09,"24,0030"
1514,1.644444e+09,"18,0250"
1518,1.644487e+09,"06,0002"


In [16]:
import numpy as np

dft = df
dft['rain'] = 0
dft

,unixTime,airHum,airTemp,wind,soilHum,soilTemp,light,(water on/off),rain
0,1626953850,25.283336,20.771570,0.119048,0.808678,9.096275,65535,NaN,0
1,1626953969,25.284093,20.773329,0.118747,0.808702,9.096275,65535,NaN,0
2,1626954030,25.286347,20.778579,0.117849,0.808775,9.097451,65535,NaN,0
3,1626954090,25.290031,20.787247,0.116365,0.808894,9.100000,65535,NaN,0
4,1626954151,25.295008,20.799230,0.114315,0.809060,9.102549,65535,NaN,0
...,...,...,...,...,...,...,...,...,...
246227,1642400795,99.900000,18.285039,39.192299,1.087440,20.835882,65535,0.0,0
246228,1642400856,99.900000,18.285434,39.187179,1.087988,20.835882,65535,0.0,0
246229,1642400917,99.900000,18.285723,39.183397,1.088382,20.835882,65535,0.0,0
246230,1642400978,99.900000,18.285899,39.181073,1.088618,20.835882,65535,0.0,0


In [17]:
for n, x in enumerate(df2['AA1']):
    dur, tot = list(map(int, x.split(',')))
    t = df2.iloc[n][0]
    t2 = t - dur*60**2
    num = len(list(dft['unixTime'].loc[(dft['unixTime'] < t) & (dft['unixTime'] > t2)]))
    if num != 0:
        avg = tot / num
        dft.loc[(dft.unixTime < t) & (dft.unixTime > t2), 'rain'] = avg
df = dft
df

,unixTime,airHum,airTemp,wind,soilHum,soilTemp,light,(water on/off),rain
0,1626953850,25.283336,20.771570,0.119048,0.808678,9.096275,65535,NaN,0.0
1,1626953969,25.284093,20.773329,0.118747,0.808702,9.096275,65535,NaN,0.0
2,1626954030,25.286347,20.778579,0.117849,0.808775,9.097451,65535,NaN,0.0
3,1626954090,25.290031,20.787247,0.116365,0.808894,9.100000,65535,NaN,0.0
4,1626954151,25.295008,20.799230,0.114315,0.809060,9.102549,65535,NaN,0.0
...,...,...,...,...,...,...,...,...,...
246227,1642400795,99.900000,18.285039,39.192299,1.087440,20.835882,65535,0.0,0.0
246228,1642400856,99.900000,18.285434,39.187179,1.087988,20.835882,65535,0.0,0.0
246229,1642400917,99.900000,18.285723,39.183397,1.088382,20.835882,65535,0.0,0.0
246230,1642400978,99.900000,18.285899,39.181073,1.088618,20.835882,65535,0.0,0.0


# Add Features 1

In [18]:
df2 = df

# Add Future soilHum
offset = df2['unixTime'][0] + 86400
# print(df2.iloc[1][1])
data = list(df2[df2['unixTime'] >= offset]['soilHum'])
data += [None for x in range(len(df2)-len(data))]
df2['future soilHum'] = data
df2

,unixTime,airHum,airTemp,wind,soilHum,soilTemp,light,(water on/off),rain,future soilHum
0,1626953850,25.283336,20.771570,0.119048,0.808678,9.096275,65535,NaN,0.0,0.792780
1,1626953969,25.284093,20.773329,0.118747,0.808702,9.096275,65535,NaN,0.0,0.792129
2,1626954030,25.286347,20.778579,0.117849,0.808775,9.097451,65535,NaN,0.0,0.791498
3,1626954090,25.290031,20.787247,0.116365,0.808894,9.100000,65535,NaN,0.0,0.790883
4,1626954151,25.295008,20.799230,0.114315,0.809060,9.102549,65535,NaN,0.0,0.790286
...,...,...,...,...,...,...,...,...,...,...
246227,1642400795,99.900000,18.285039,39.192299,1.087440,20.835882,65535,0.0,0.0,NaN
246228,1642400856,99.900000,18.285434,39.187179,1.087988,20.835882,65535,0.0,0.0,NaN
246229,1642400917,99.900000,18.285723,39.183397,1.088382,20.835882,65535,0.0,0.0,NaN
246230,1642400978,99.900000,18.285899,39.181073,1.088618,20.835882,65535,0.0,0.0,NaN


# Remove Anomalies

In [19]:
dropRows = df[df['wind'] < 0].index
for col in list(df2.columns):
    df2[col] = df[col].drop(dropRows, inplace = False)
df=df2
df['(water on/off)'] = df['(water on/off)'].fillna(0)
df

,unixTime,airHum,airTemp,wind,soilHum,soilTemp,light,(water on/off),rain,future soilHum
0,1.626954e+09,25.283336,20.771570,0.119048,0.808678,9.096275,65535.0,0.0,0.0,0.792780
1,1.626954e+09,25.284093,20.773329,0.118747,0.808702,9.096275,65535.0,0.0,0.0,0.792129
2,1.626954e+09,25.286347,20.778579,0.117849,0.808775,9.097451,65535.0,0.0,0.0,0.791498
3,1.626954e+09,25.290031,20.787247,0.116365,0.808894,9.100000,65535.0,0.0,0.0,0.790883
4,1.626954e+09,25.295008,20.799230,0.114315,0.809060,9.102549,65535.0,0.0,0.0,0.790286
...,...,...,...,...,...,...,...,...,...,...
246227,1.642401e+09,99.900000,18.285039,39.192299,1.087440,20.835882,65535.0,0.0,0.0,NaN
246228,1.642401e+09,99.900000,18.285434,39.187179,1.087988,20.835882,65535.0,0.0,0.0,NaN
246229,1.642401e+09,99.900000,18.285723,39.183397,1.088382,20.835882,65535.0,0.0,0.0,NaN
246230,1.642401e+09,99.900000,18.285899,39.181073,1.088618,20.835882,65535.0,0.0,0.0,NaN


# Interpolate

In [20]:
df = df.interpolate(method='nearest',limit=30)

# Add Features 2

In [21]:
df2 = df

# Add time after last reading
times = tuple(df2['unixTime'])
data = [0] + [times[n]-times[n-1] for n in range(1, len(times))]
# df2.drop(columns='Gap', inplace=True)
df2.insert(1, 'Gap', data)
df = df2
df

,unixTime,Gap,airHum,airTemp,wind,soilHum,soilTemp,light,(water on/off),rain,future soilHum
0,1.626954e+09,0.0,25.283336,20.771570,0.119048,0.808678,9.096275,65535.0,0.0,0.0,0.792780
1,1.626954e+09,119.0,25.284093,20.773329,0.118747,0.808702,9.096275,65535.0,0.0,0.0,0.792129
2,1.626954e+09,61.0,25.286347,20.778579,0.117849,0.808775,9.097451,65535.0,0.0,0.0,0.791498
3,1.626954e+09,60.0,25.290031,20.787247,0.116365,0.808894,9.100000,65535.0,0.0,0.0,0.790883
4,1.626954e+09,61.0,25.295008,20.799230,0.114315,0.809060,9.102549,65535.0,0.0,0.0,0.790286
...,...,...,...,...,...,...,...,...,...,...,...
246227,1.642401e+09,60.0,99.900000,18.285039,39.192299,1.087440,20.835882,65535.0,0.0,0.0,NaN
246228,1.642401e+09,61.0,99.900000,18.285434,39.187179,1.087988,20.835882,65535.0,0.0,0.0,NaN
246229,1.642401e+09,61.0,99.900000,18.285723,39.183397,1.088382,20.835882,65535.0,0.0,0.0,NaN
246230,1.642401e+09,61.0,99.900000,18.285899,39.181073,1.088618,20.835882,65535.0,0.0,0.0,NaN


In [ ]:
# get hour and day mean, median, max, min and std dev

dft = df

hour = 60**2
day = 24*60**2

# print(dft.columns.get_loc('unixTime'))

for col in dft.columns[1:-1]:
    for x in ['mean', 'median', 'max', 'min', 'std dev']:
        for time in ['hourly', 'daily']:
            dataFinal[' '.join([col, time, x])] = []
print(dataFinal)
            
for r in range(dft.shape[0]):
    curTime = float(dft.iloc[[r]]['unixTime'])
    dataH = dft[(dft['unixTime'] <= curTime) & (dft['unixTime'] >= curTime-hour)]
    dataD = dft[(dft['unixTime'] <= curTime) & (dft['unixTime'] >= curTime-day)]
    for col in dataH.columns[1:-1]:
        ins = col + ' hourly'
        dataFinal[ins + ' mean'].append(np.mean(dataH[col]))
        dataFinal[ins + ' median'].append(np.median(dataH[col]))
        dataFinal[ins + ' max'].append(np.max(dataH[col]))
        dataFinal[ins + ' min'].append(np.min(dataH[col]))
        dataFinal[ins + ' std dev'].append(np.std(dataH[col]))
        
    for col in dataD.columns[1:-1]:
        ins = col + ' daily'
        dataFinal[ins + ' mean'].append(np.mean(dataD[col]))
        dataFinal[ins + ' median'].append(np.median(dataD[col]))
        dataFinal[ins + ' max'].append(np.max(dataD[col]))
        dataFinal[ins + ' min'].append(np.min(dataD[col]))
        dataFinal[ins + ' std dev'].append(np.std(dataD[col]))
    print('finished row', r+1, 'of', dft.shape[0])

In [ ]:
# merge new data

for newCol in dataFinal:
    if newCol in dft:
        dft.drop(columns=newCol, inplace=True)
    if newCol.split(' ')[0] in dft:
        oriCol = newCol.split(' ')[0]
    else:
        oriCol = ' '.join(newCol.split(' ')[:2])
    dft.insert(dft.columns.get_loc(oriCol), newCol, dataFinal[newCol])
    print(newCol)

In [215]:
df.to_csv('features.csv')